In [ ]:
import ipywidgets as widgets
from IPython.display import display
from pathlib import Path
import leafmap.leafmap as leafmap  # ipyleaflet backend

DATA_DIR = Path("/gee_out").resolve()

RASTER_EXTS = (".tif", ".tiff", ".img")
VECTOR_EXTS = (".gpkg", ".shp", ".geojson", ".json")


def find_products():
    """Recursively search /gee_out for rasters and vectors."""
    files = []
    if not DATA_DIR.exists():
        return files

    for ext in (*RASTER_EXTS, *VECTOR_EXTS):
        files.extend(sorted(DATA_DIR.rglob(f"*{ext}")))
    return files


def make_dropdown_options():
    files = find_products()
    if not files:
        # empty dropdown
        return [("No products found", None)]

    # Show paths relative to /gee_out, keep absolute path as value
    return [
        (str(p.relative_to(DATA_DIR)), str(p))
        for p in files
    ]


# Create Leafmap map centered on Argentina
m = leafmap.Map(
    center=[-38, -64],
    zoom=4,
    toolbar_ctrl=True,
    draw_ctrl=True,
    measure_ctrl=True,
    fullscreen_ctrl=True,
)

info = widgets.HTML(
    """<h3>Leafmap viewer</h3>
    <ul>
      <li>Use the <b>dropdown</b> and buttons below to load GeoTIFFs and vectors found under <code>/gee_out</code>.</li>
      <li>Click <b>Refresh list</b> after the chatbot generates new products.</li>
      <li>You can also use the Leafmap toolbar (Raster / Vector GUI) to load and style files manually.</li>
    </ul>
    """
)

product_dropdown = widgets.Dropdown(
    options=make_dropdown_options(),
    description="Product:",
    layout=widgets.Layout(width="500px"),
)

refresh_button = widgets.Button(
    description="Refresh list",
    icon="refresh",
    tooltip="Rescan /gee_out for new products",
)

add_button = widgets.Button(
    description="Add to map",
    icon="plus",
    tooltip="Add selected product to the map",
)

clear_button = widgets.Button(
    description="Clear layers",
    icon="trash",
    tooltip="Remove loaded layers (keep basemap)",
)

status_label = widgets.HTML("")


def refresh_list(_=None):
    product_dropdown.options = make_dropdown_options()
    status_label.value = "<span style='color:gray'>Product list refreshed.</span>"


def clear_layers(_=None):
    # keep only the first layer (basemap)
    try:
        m.layers = m.layers[:1]
    except Exception:
        pass
    status_label.value = "<span style='color:gray'>Layers cleared (basemap kept).</span>"


def add_selected(_=None):
    value = product_dropdown.value
    if not value:
        status_label.value = "<span style='color:red'>No product selected.</span>"
        return

    path = Path(value)
    if not path.exists():
        status_label.value = f"<span style='color:red'>File not found: {path}</span>"
        return

    ext = path.suffix.lower()
    try:
        if ext in RASTER_EXTS:
            # Let Leafmap handle band / min / max / colormap; user can tweak via Raster GUI.
            m.add_raster(
                str(path),
                layer_name=path.name,
            )
            status_label.value = f"<span style='color:green'>Raster added: {path.name}</span>"
        elif ext in VECTOR_EXTS:
            m.add_vector(
                str(path),
                layer_name=path.name,
            )
            status_label.value = f"<span style='color:green'>Vector added: {path.name}</span>"
        else:
            status_label.value = f"<span style='color:red'>Unsupported file type: {path.suffix}</span>"
    except Exception as e:
        status_label.value = f"<span style='color:red'>Error adding layer: {e}</span>"


refresh_button.on_click(refresh_list)
add_button.on_click(add_selected)
clear_button.on_click(clear_layers)

controls = widgets.HBox(
    [product_dropdown, refresh_button, add_button, clear_button],
    layout=widgets.Layout(margin="5px 0px"),
)

# Attach Leafmap's own GUI panels (optional, but gives you band / min-max / colormap controls).
try:
    leafmap.raster_gui(m, data_dir=str(DATA_DIR))
    leafmap.vector_gui(m, data_dir=str(DATA_DIR))
except Exception:
    # If these are unavailable in your leafmap version, you still have the map + dropdown.
    pass

display(info)
display(controls)
display(status_label)
display(m)

# Populate the initial list once
refresh_list()